In [1]:
import torch
from trainer import Trainer
from dataset import YoloVOCDataset
from model import YoloV3
from loss import YoloV3Loss
from utils import plot_labels, plot_prediction
import config
from torch.utils.data import DataLoader
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from pprint import pprint


/Users/joaofernandes/.pyenv/versions/3.10.5/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = YoloVOCDataset(
    csv_file=config.DATASET_PATH + "2examples.csv",
    image_dir=config.IMAGES_PATH,
    label_dir=config.LABELS_PATH,
    transform=config.test_transform,
)
loader = DataLoader(dataset, batch_size=2, num_workers=2)
model = YoloV3(20).to(config.DEVICE)
loss_fn = YoloV3Loss().to(config.DEVICE)
model = YoloV3(20).to(config.DEVICE)
optimizer = torch.optim.Adam
epochs = 500
lr = 0.001
batch_size = 2
checkpoint_path = config.CHECKPOINT_PATH
scaled_anchors = (
    torch.tensor(config.ANCHORS)
    * torch.tensor(
        [config.IMAGE_SIZE // 32, config.IMAGE_SIZE // 16, config.IMAGE_SIZE // 8]
    )
    .unsqueeze(1)
    .unsqueeze(1)
    .repeat(1, 3, 2)
).to(config.DEVICE)

trainer = Trainer(
    model,
    loader,
    loss_fn,
    scaled_anchors,
    optimizer,
    epochs,
    lr,
    batch_size,
    checkpoint_path,
)
trainer.load_checkpoint(checkpoint_path)
images, labels = next(iter(loader))

model.eval()
predictions = model(images.to(config.DEVICE))
# plot_labels(images, labels, 1)


/Users/joaofernandes/.pyenv/versions/3.10.5/lib/python3.10/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


Loaded checkpoint trained on 500 epochs with a loss of 0.018840786069631577


In [3]:
import importlib
imported_module = importlib.import_module("utils")
importlib.reload(imported_module)
from utils import plot_labels, plot_prediction, get_mAP

# plot_prediction(images, predictions, 1)
# plot_labels(images, labels, 1)

# mAP = MeanAveragePrecision(box_format="cxcywh")
# mAP.update(preds, target)
# pprint(mAP.compute())

pprint(get_mAP(predictions, labels))

{'map': tensor(0.9667),
 'map_50': tensor(1.),
 'map_75': tensor(1.),
 'map_large': tensor(-1.),
 'map_medium': tensor(-1.),
 'map_per_class': tensor(-1.),
 'map_small': tensor(0.9667),
 'mar_1': tensor(0.7444),
 'mar_10': tensor(0.9667),
 'mar_100': tensor(0.9667),
 'mar_100_per_class': tensor(-1.),
 'mar_large': tensor(-1.),
 'mar_medium': tensor(-1.),
 'mar_small': tensor(0.9667)}
